In [1]:
!pip install sklearn-pandas

  Using cached https://files.pythonhosted.org/packages/c5/9e/42d7bcddb09a3ff52d0c60c810ba5d0fded28abbe320c85bbf7368192956/sklearn_pandas-1.6.0-py2.py3-none-any.whl


In [2]:
from datascience import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')

# import various tools from `sklearn`
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import sklearn.cluster as skcluster
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from sklearn_pandas import DataFrameMapper
from collections import Counter

# Initial forays into the canonicalization problem

As with any problem where you start off having no idea what to do, exploratory data analysis is required.

In [26]:
dat = pd.read_json('../data/test_data.json')
metadata = pd.DataFrame(list(dat['metadata']))
metadata

,annotators,article_number,city,date_published,filename,periodical,periodical_code,state,version
0,[CRV],1368,Charleston,2011-11-23,"1368Charleston, SC-live5news-03.txt",live5news,03,SC,27
1,[NBA],7331,Seattle,2011-10-29,7331Seattle-King5_Seattle-01.txt,King5_Seattle,01,None,30
2,"[CRV, EDC]",1008,Bethlehem,2011-12-08,"1008Bethlehem, PA-theassociatedpressstatelocal...",theassociatedpressstatelocalwire,04,PA,26
3,"[CD, V]",1767,Cleveland,None,"1767Cleveland, TN-ClevelandBanner-01.txt",ClevelandBanner,01,TN,None
4,"[EDC, MB]",912,Baltimore,2011-12-05,912Baltimore-TheBaltimoreSun-01.txt,TheBaltimoreSun,01,None,26
5,[SLG],202,Albany,2011-12-23,"202Albany, NY-TimesUnion-04.txt",TimesUnion,04,NY,26
6,[CRV],2335,Denver,2011-12-12,2335Denver-theassociatedpressstatelocalwire-20...,theassociatedpressstatelocalwire,20,None,31
7,[CRV],3023,Huntington,2011-12-01,3023Huntington-theassociatedpressstatelocalwir...,theassociatedpressstatelocalwire,03,None,28
8,[SLG],4008,Lubbock,2011-11-15,4008Lubbock-KBCD-03.txt,KBCD,03,None,29
9,[CRV],3638,LosAngeles,2011-12-03,3638LosAngeles-LATimes-05.txt,LATimes,05,None,29


In [36]:
city_and_date = pd.DataFrame(list(zip(metadata['city'], metadata['date_published'])))
text_and_id = pd.DataFrame(list(zip(dat['id'], dat['text'])))
concat_tbl = pd.concat([city_and_date, text_and_id], axis = 1) 
concat_tbl.columns = ['city', 'date', 'id', 'text']

In [38]:
reading_set = Table().from_df(concat_tbl)
reading_set

city,date,id,text
Charleston,2011-11-23,19,Occupy demonstrators to be released on bond Posted: Nov ...
Seattle,2011-10-29,193,by NATASHA RYAN / KING 5 News Bio | Email Posted on Oct ...
Bethlehem,2011-12-08,2,3 of 4 DOCUMENTS The Associated Press State & Local Wire ...
Cleveland,None,28,"6 protesters form Occupy Cleveland by DAVID DAVIS, Manag ..."
Baltimore,2011-12-05,220,"Woman stabbed at Occupy Baltimore encampment, police say ..."
Albany,2011-12-23,34,"Dismay, hard feelings after confrontation City leaders d ..."
Denver,2011-12-12,43,134 of 156 DOCUMENTS The Associated Press State & Local ...
Huntington,2011-12-01,65,3 of 6 DOCUMENTS The Associated Press State & Local Wire ...
Lubbock,2011-11-15,97,KBCD: http://www.kcbd.com/ Mayor says Occupy Lubbock pro ...
LosAngeles,2011-12-03,84,Occupy L.A. protesters return to downtown streets Decemb ...


In [43]:
charleston = reading_set.where("city", "Charleston")
charleston.with_column('event')

city,date,id,text
Charleston,2011-11-23,19,Occupy demonstrators to be released on bond Posted: Nov ...
Charleston,2011-10-22,16,Occupy Charleston takes over Brittlebank Park for 99 hou ...
Charleston,2011-10-20,15,"Occupy Charleston holds march Posted: Oct 15, 2011 1:40 ..."
Charleston,2011-10-28,20,Link: http://www.wvgazette.com/News/201110190148 October ...
Charleston,2011-10-19,18,"12 of 14 DOCUMENTS The Associated Press October 19, 2011 ..."


In [45]:
event_ids = make_array(1, 2, 3, 4, 5)
charleston = charleston.with_column('event_ids', event_ids)

In [46]:
charleston

city,date,id,text,event_ids
Charleston,2011-11-23,19,Occupy demonstrators to be released on bond Posted: Nov ...,1
Charleston,2011-10-22,16,Occupy Charleston takes over Brittlebank Park for 99 hou ...,2
Charleston,2011-10-20,15,"Occupy Charleston holds march Posted: Oct 15, 2011 1:40 ...",3
Charleston,2011-10-28,20,Link: http://www.wvgazette.com/News/201110190148 October ...,4
Charleston,2011-10-19,18,"12 of 14 DOCUMENTS The Associated Press October 19, 2011 ...",5
